# Facebook AI Similarity Search (FAISS)

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("./speech.txt")
documents=loader.load()
documents

text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=30)

docs=text_splitter.split_documents(documents)

Created a chunk of size 668, which is longer than the specified 500
Created a chunk of size 982, which is longer than the specified 500
Created a chunk of size 789, which is longer than the specified 500


In [8]:
docs

[Document(metadata={'source': './speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': './speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': './speech.txt'}, p

In [13]:
embeddings=OllamaEmbeddings(model="phi4-mini:latest")
db=FAISS.from_documents(docs,embeddings)
db

In [16]:
### querying
query="what does the speaker belive in the main reason the united state should enter the war?"

docs=db.similarity_search(query)

print(docs[0].page_content)
docs

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.

…


[Document(id='0fef1a21-f9a3-4635-ae4b-0bccb4b35a6e', metadata={'source': './speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='8e5af517-9aab-4196-b203-dab0f2d455d6', metadata={'source': './speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='21b2d20c-982d-4395-b0ba-207506bfc89c', metadata={'source': './s

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'

# As a retriever
### we can also converts the vectorstores into a Retriever class.this allows us to easily use it in other langchain methods, which largely work with retrievers

In [19]:
retriever=db.as_retriever()
docs=retriever.invoke(query)

docs[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'

# Similarity search with score
### uses manhatten distance (L2)

In [24]:
docs_and_scores=db.similarity_search_with_score(query)

print(docs_and_scores[0][0].page_content)
print(docs_and_scores[0][1])

docs_and_scores

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.

…
1418.8302


[(Document(id='0fef1a21-f9a3-4635-ae4b-0bccb4b35a6e', metadata={'source': './speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
  np.float32(1418.8302)),
 (Document(id='8e5af517-9aab-4196-b203-dab0f2d455d6', metadata={'source': './speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  np.float32(1921.9039)),
 (Document(id='21b2d20c

In [25]:
embeddings_vectors=embeddings.embed_query(query)

docs_scores=db.similarity_search_by_vector(embeddings_vectors)

print(docs_scores[0].page_content)
docs_scores

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.

…


[Document(id='0fef1a21-f9a3-4635-ae4b-0bccb4b35a6e', metadata={'source': './speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='8e5af517-9aab-4196-b203-dab0f2d455d6', metadata={'source': './speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='21b2d20c-982d-4395-b0ba-207506bfc89c', metadata={'source': './s

# saving and loading

In [28]:
db.save_local("faiss_index")

In [33]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
new_db

In [ ]:
docs=new_db.similarity_search("query")
docs[0].page_content

[Document(id='0fef1a21-f9a3-4635-ae4b-0bccb4b35a6e', metadata={'source': './speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='0f87b4ca-e39d-4f42-bb15-596eb5e944db', metadata={'source': './speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things 